In [1]:
import stanza
import nltk
import time
import math
import collections
import json
from nltk.corpus import movie_reviews
from numpy import dot
from numpy.linalg import norm

In [2]:
#stanza.download('en')

2020-12-05 20:32:24 INFO: Downloading default packages for language: en (English)...
2020-12-05 20:45:38 INFO: Finished downloading models and saved to C:\Users\1111\stanza_resources.


In [2]:
%config IPCompleter.greedy=True

In [3]:
nlp = stanza.Pipeline('en')

2020-12-10 08:41:47 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

2020-12-10 08:41:47 INFO: Use device: cpu
2020-12-10 08:41:47 INFO: Loading: tokenize
2020-12-10 08:41:48 INFO: Loading: pos
2020-12-10 08:41:53 INFO: Loading: lemma
2020-12-10 08:41:53 INFO: Loading: depparse
2020-12-10 08:41:56 INFO: Loading: sentiment
2020-12-10 08:42:02 INFO: Loading: ner
2020-12-10 08:42:08 INFO: Done loading processors!


In [41]:
#nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\1111\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


True

In [4]:
def get_input_vector(input_text, merged_dict):
    input_nlp = nlp(input_text)
    input_dictionary = dict()
    input_dictionary = add_dependencies_to_dictionary(input_dictionary, input_nlp)
    cleaned_input_dictionary = clean_input_from_insufficient(input_dictionary, merged_dict)
    completed_input_dictionary = complete_dictionary_with_opposite(cleaned_input_dictionary, merged_dict)
    sorted_input_dictionary = sort_dictionary(completed_input_dictionary)
    normalized_input_dictionary = normalize_dictionary(sorted_input_dictionary)
    input_vector = get_vector_from_dict(normalized_input_dictionary)
    return input_vector

def preprocessing():
    positive_dictionary = dict()
    negative_dictionary = dict()
    positive_fileids = movie_reviews.fileids('pos')
    negative_fileids = movie_reviews.fileids('neg')
    print('POSITIVE REVIEWS TRAIN SET LENGTH: ', len(positive_fileids))
    print('NEGATIVE REVIEWS TRAIN SET LENGTH: ', len(negative_fileids))
    positive_dictionary = get_dependencies_dictionary(positive_fileids[0:500], positive_dictionary)
    negative_dictionary = get_dependencies_dictionary(negative_fileids[0:500], negative_dictionary)
    positive_completed = complete_dictionary_with_opposite(positive_dictionary, negative_dictionary)
    negative_completed = complete_dictionary_with_opposite(negative_dictionary, positive_dictionary)
    merged_dictionary = merge_dictionaries(positive_completed, negative_completed)
    merged_sorted_dictionary = sort_dictionary(merged_dictionary)
    positive_divided = divide_dictionary_by_summary(positive_completed, merged_dictionary)
    negative_divided = divide_dictionary_by_summary(negative_completed, merged_dictionary)
    positive_normalized = normalize_dictionary(positive_divided)
    negative_normalized = normalize_dictionary(negative_divided)
    positive_sorted_normalized = sort_dictionary(positive_normalized)
    negative_sorted_normalized = sort_dictionary(negative_normalized)
    print('POSITIVE SORTED NORMALIZED DEPENDENCIES DICTIONARY: ', positive_sorted_normalized)
    print('NEGATIVE SORTED NORMALIZED DEPENDENCIES DICTIONARY: ', negative_sorted_normalized)
    positive_vector = get_vector_from_dict(positive_sorted_normalized)
    negative_vector = get_vector_from_dict(negative_sorted_normalized)
    return merged_sorted_dictionary, positive_vector, negative_vector

def load_saved_preprocessing():
    f = open("json_merged_dict.txt", "r")
    json_merged = f.read()
    f.close()
    f = open("json_positive_vector.txt", "r")
    json_positive = f.read()
    f.close()
    f = open("json_negative_vector.txt", "r")
    json_negative = f.read()
    f.close()
    merged_dictionary_r = json.loads(json_merged)
    positive_vector_read = json.loads(json_positive)["items"]
    negative_vector_read = json.loads(json_negative)["items"]
    positive_vector_r = json.loads(positive_vector_read)
    negative_vector_r = json.loads(negative_vector_read)
    merged_sorted_dictionary_r = sort_dictionary(merged_dictionary_r)
    return merged_sorted_dictionary_r, positive_vector_r, negative_vector_r

def save_preprocessing(merged_sorted_dictionary, positive_vector, negative_vector):
    json_merged_dict = json.dumps(merged_sorted_dictionary)
    upload_positive_dict = dict()
    upload_positive_dict["items"] = json.dumps(positive_vector)
    json_positive_vector = json.dumps(upload_positive_dict)
    upload_negative_dict = dict()
    upload_negative_dict["items"] = json.dumps(negative_vector)
    json_negative_vector = json.dumps(upload_negative_dict)
    f = open("json_merged_dict.txt", "a")
    f.write(json_merged_dict)
    f.close()
    f = open("json_positive_vector.txt", "a")
    f.write(json_positive_vector)
    f.close()
    f = open("json_negative_vector.txt", "a")
    f.write(json_negative_vector)
    f.close()

def get_dependencies_dictionary(fileids, dictionary):
    count = 0        
    print('Prerocessing started.')
    for fileid in fileids:
        print('Reading file No.: ', count)
        stream = movie_reviews.open(fileid)
        review = stream.read()
        review_nlp = nlp(review)
        dictionary = add_dependencies_to_dictionary(dictionary, review_nlp)
        count += 1
    return dictionary
        
def add_dependencies_to_dictionary(dictionary, document_nlp):
    for sentence in document_nlp.sentences:
        for dependency in sentence.dependencies:
            if dependency[1] == 'punct' or dependency[1] == 'root':
                continue
            else:
                key = dependency[0].lemma + '_' + dependency[1] + '_' + dependency[2].lemma
                if key in dictionary:
                    dictionary[key] += 1
                else:
                    dictionary[key] = 1
    return dictionary
    
def print_dependencies(document_nlp):
    for sentence in document_nlp.sentences:
        for dependency in sentence.dependencies:
            if dependency[1] == 'punct' or dependency[1] == 'root':
                continue
            if isinstance(dependency[0].lemma, str):
                print(dependency[0].lemma + ' : ' + dependency[1] + ' : ' + dependency[2].lemma)
            else:
                print(dependency[0].text + ' : ' + dependency[1] + ' : ' + dependency[2].lemma)
    return document_nlp

def complete_dictionary_with_opposite(target, opposite):
    for key, value in opposite.items():
        if not key in target:
            target[key] = 0
    return target

def merge_dictionaries(dictionary1, dictionary2):
    merged_dictionary = dict()
    for key, value in dictionary1.items():
        merged_dictionary[key] = dictionary1[key] + dictionary2[key]
    return merged_dictionary
            
def divide_dictionary_by_summary(target, summary):
    target_result = dict(target)
    for key, value in target_result.items():
        target_result[key] = target_result[key] / summary[key]
    return target_result

def normalize_dictionary(dictionary):
    result_normalized = dict(dictionary)
    sum_of_squares = 0
    for key, value in result_normalized.items():
        sum_of_squares += (value * value)
    print('sum_of_squares: ', sum_of_squares)
    for key, value in result_normalized.items():
        result_normalized[key] = result_normalized[key] / math.sqrt(sum_of_squares)
    return result_normalized

def sort_dictionary(dictionary):
    ordered_dictionary = collections.OrderedDict(sorted(dictionary.items()))
    return ordered_dictionary

def get_vector_from_dict(dictionary):
    vector = list()
    for key, value in dictionary.items():
        vector.append(value)
    return vector

def clean_input_from_insufficient(input_dictionary, merged_dict):
    result_input = dict(input_dictionary)
    for key, value in input_dictionary.items():
        if not key in merged_dict:
            del result_input[key]
    return result_input

def predict(input_vector, positive_vector, negative_vector):
    cos_with_positive = dot(input_vector, positive_vector)/(norm(input_vector)*norm(positive_vector))
    cos_with_negative = dot(input_vector, negative_vector)/(norm(input_vector)*norm(negative_vector))
    print('COS with POSITIVE: ', cos_with_positive)
    print('COS with NEGATIVE: ', cos_with_negative)
    if cos_with_positive > cos_with_negative:
        return 'positive'
    else:
        return 'negative'

In [212]:
#start_time = time.clock()
#merged_sorted_dictionary, positive_vector, negative_vector = preprocessing()
#end_time = time.clock()

In [5]:
merged_sorted_dictionary, positive_vector, negative_vector = load_saved_preprocessing()

In [216]:
#preprocessing lasted for 6 hours. 
print('Result dictionary length: ', len(merged_sorted_dictionary))

Result dictionary length:  338829


In [18]:
merged_sorted_dictionary

OrderedDict([('\x05euuugh_case_por', 1),
             ('\x13_acl:relcl_lie', 1),
             ('\x13_advmod_somewhat', 1),
             ('\x13_det_these', 1),
             ('\x13suspend_advcl_enjoy', 1),
             ('\x13suspend_aux_must', 1),
             ('\x13suspend_mark_that', 1),
             ('\x13suspend_nsubj_you', 1),
             ('\x13suspend_obj_disbelief', 1),
             ('\x16_advcl_way', 1),
             ('\x16_advmod_far', 1),
             ('\x16_amod_perfect', 1),
             ('\x16_case_from', 1),
             ('\x16_conj_look', 1),
             ('\x16_cop_be', 1),
             ('\x16_nsubj_it', 1),
             ('#_advmod_now', 1),
             ('#_appos_blade', 1),
             ('#_case_after', 1),
             ('#_case_through', 1),
             ('#_cc_and', 1),
             ('#_compound_comparison', 2),
             ('#_compound_novikov&', 4),
             ('#_compound_theater', 4),
             ('#_nmod_$', 1),
             ('#_nmod_brother', 1),
          

In [7]:
positive_vector

[0.002438494207049591,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.002438494207049591,
 0.002438494207049591,
 0.0,
 0.002438494207049591,
 0.002438494207049591,
 0.002438494207049591,
 0.0012192471035247954,
 0.001828870655287193,
 0.002438494207049591,
 0.002438494207049591,
 0.002438494207049591,
 0.002438494207049591,
 0.0009144353276435965,
 0.0,
 0.0012192471035247954,
 0.0,
 0.001828870655287193,
 0.002438494207049591,
 0.002438494207049591,
 0.002438494207049591,
 0.002438494207049591,
 0.002438494207049591,
 0.002438494207049591,
 0.0,
 0.002438494207049591,
 0.001625662804699727,
 0.0,
 0.0012192471035247954,
 0.0,
 0.002438494207049591,
 0.0,
 0.002438494207049591,
 0.002438494207049591,
 0.002438494207049591,
 0.0,
 0.0,
 0.002438494207049591,
 0.002438494207049591,
 0.002438494207049591,
 0.001625662804699727,
 0.002438494207049591,
 0.0017417815764639935,
 0.0,
 0.002438494207049591,
 0.002438494207049591,
 0.00243849420704

In [8]:
negative_vector

[0.0,
 0.00255027661283171,
 0.00255027661283171,
 0.00255027661283171,
 0.00255027661283171,
 0.00255027661283171,
 0.00255027661283171,
 0.00255027661283171,
 0.00255027661283171,
 0.00255027661283171,
 0.00255027661283171,
 0.00255027661283171,
 0.00255027661283171,
 0.00255027661283171,
 0.00255027661283171,
 0.00255027661283171,
 0.0,
 0.0,
 0.00255027661283171,
 0.0,
 0.0,
 0.0,
 0.001275138306415855,
 0.0006375691532079275,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0015939228830198187,
 0.00255027661283171,
 0.001275138306415855,
 0.00255027661283171,
 0.0006375691532079275,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.00255027661283171,
 0.0,
 0.0008500922042772366,
 0.00255027661283171,
 0.001275138306415855,
 0.00255027661283171,
 0.0,
 0.00255027661283171,
 0.0,
 0.0,
 0.0,
 0.00255027661283171,
 0.00255027661283171,
 0.0,
 0.0,
 0.0,
 0.0008500922042772366,
 0.0,
 0.0007286504608090599,
 0.00255027661283171,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.00255027661283171,
 0.00255027661283171,
 0.0025502766

In [9]:
test_positive = '''Martin Scorsese has done it again. His newest and most refreshing effort he's contributed to the world of cinema in years, The Wolf of Wall Street is a roaring thrill ride that is both absolutely hilarious and meticulously constructed. It also presents Academy Award nominee Leonardo DiCaprio in possibly his finest acting performance of his career. At one-minute shy of three hours, I was both engaged and hypnotized nearly the entire duration. A comedic epic that studies the behavior and cultures of a time in America, feels like the uncovering of a time capsule that was buried and dug up to give insight into our current financial crisis. Much more than just laughs, it turns on the dramatic elements early enough in the film to warrant considerable reactions about the choices of our key characters. Expertly paced with intelligent moral questions presented, The Wolf of Wall Street is one of the best films of the year.

Telling the story of Jordan Belfort, a young Wall Street broker that gets involved in drugs, money, and even more drugs during the 80's and 90's. In his tenure trading (and stealing), Jordan marries, divorces, does drugs, marries again, does even more drugs, makes solid friendships, and believe it or not, does a lot more drugs. Watching the destruction of Jordan acted as a documentarian's insight that felt like I was watching "Intervention" without the family that cares. The Wolf of Wall Street is a black comedy, giving hints of drama. Natural comparisons will fly to Oliver Stone's Wall Street which is accurate but you can see subtle hints of films like Trading Places, Glengarry Glen Ross, and even American Psycho. That's a testament to Scorsese's outstanding direction and Terence Winter's masterful screenplay. Scorsese keeps Wolf life-size, sprinkled with characters that are both geniuses and morons, but functioning morons. They're like the frat pack group that sat in a corner on my college campus, being loud and obnoxious, and made terrible life choices that they still aren't aware of until this day. Scorsese puts together an all-star cast to inhabit these beings that includes DiCaprio, Jonah Hill, Margot Robbie, Matthew McConaughey, Jon Bernathal, and Kyle Chandler. All of which seem to be having the time of their lives.

A lot of the credit of the film's overall success has to be awarded to Leonardo DiCaprio. I've never seen him truly "go for it" in a way that he exhibits as Jordan Belfort. In his breaking of the fourth wall, to his long but completely engaging monologues about life, money, and greed, it's the most assured and compelling work by the actor to date. When DiCaprio unleashed his talents in the mid-90's in What's Eating Gilbert Grape? and later stole the hearts of tween girls everywhere in Titanic, who knew this is the role he'd been gearing up to play. This is the role of his career and something that the Academy Awards should look to for his long overdue recognition. It's a charming and adventurous turn that presents a conundrum to the audience as we find ourselves both enamored and loathing the pure essence of Jordan. A sequence of DiCaprio crawling on the floor will probably be the scene of the year. This is DiCaprio's crowning achievement.

As the magnetic and cheesy-minded right-hand man, Jonah Hill's performance as Donnie Azoff is another great turn for the 30-year-old actor. He's allowed to explore some of his comedic ticks and beats that he may not have ever had the opportunity to explore in films like Superbad or 21 Jump Street. In Wolf, he relies on his own instincts, and his chemistry with DiCaprio, which has helped him before for his Oscar-nominated work in Moneyball opposite Brad Pitt. Matthew McConaughey, is one scene shy of winning the Oscar for Best Supporting Actor. While his work in Dallas Buyers Club will bring him the acclaim and recognition that he deserves, The Wolf of Wall Street is a prime example of what he should be doing when he's not working or seeking out the strong, independent features that are geared for awards recognition. Stealing every frame and focus from DiCaprio in his ten minute screen time, McConaughey utilizes all his charm and spunk as Mark Hanna, the mentor to young Jordan as he started out.

Like any great Scorsese film, the women are in full-force and given the opportunity to shine like the others. Cristin Millotti, a toned down and tragic version of Marisa Tomei in My Cousin Vinny, is sensational in her brief appearances on screen. Beautiful and sympathetic, she offers much needed serious and dramatic elements to Jordan's outrageous antics. In the end, a star is born in the gorgeous and vivacious Margot Robbie as Naomi Lapaglia, Jordan's second wife. Whoever was going to be cast as Naomi, had to be an actress of considerable talent and had the ability to really be the sexy kitten but still warrant an emotional reaction from the audience when called upon. Margot Robbie was the perfect choice and she'll need to owe Scorsese royalties for years to come with the roles she'll be offered following this. Robbie is pure magic and is everything she's required to be. She's the more elusive, compelling, and more thought out version of Scarlett Johansson's character in Don Jon.

I loved every second of The Wolf of Wall Street. Terence Winter's script is a natural and well-oiled machine that produces the words of a demigod. You couldn't make these things up. Thelma Schoonmaker is the utmost professional and continues to shine film after film. You won't find another dedicated and glossed editing work this year. The other supporting actors do sensational work especially Kyle Chandler, who has a very well-constructed exchange on a boat with DiCaprio, has us asking more and more, why is this guy not helming his own films on a consistent basis yet?
'''

In [10]:
test_negative = '''Even though Jordan Belfort is a convicted con man, Scorsese & DiCaprio were apparently too dense to realize that his book was yet another of his scams. They leapt on Belfort's book like pigs on a pile of slop and thought everyone would be just as enamored as they were with the alleged life story of a sociopathic, sleazeball swindler. One example of how they got conned: In real life, Belfort never warned his cohorts that he was wearing a wire! In reality, he was a cry-baby snitch who immediately agreed to testify against his co-conspirators, never giving any of them a word of warning.

It's rather amusing to watch Scorsese & DiCaprio as they try to explain that this movie isn't a glorification of Belfort's swindling & debauchery. C'mom, guys, 'fess up, all three hours of the movie is a celebration of it! You wasted $100 million to make an homage to a drug- addicted, misogynistic low-life who swindled people out of their hard- earned money! Proud of yourselves?

Here's the "Spoiler Alert": You are being conned if you go to this movie. But go ahead and give your hard-earned money to this convicted swindler and the "Hollywood sophisticates" who think his pathetically sad life is worth being turned into a movie. Just know that Belfort is laughing all the way to the bank, and you are his latest victim.'''

In [11]:
positive_input_vector = get_input_vector(test_positive, merged_sorted_dictionary)
negative_input_vector = get_input_vector(test_negative, merged_sorted_dictionary)

sum_of_squares:  503
sum_of_squares:  121


In [12]:
predict(positive_input_vector, positive_vector, negative_vector)

COS with POSITIVE:  0.026624425801600717
COS with NEGATIVE:  0.022529158275006605


'positive'

In [13]:
predict(negative_input_vector, positive_vector, negative_vector)

COS with POSITIVE:  0.01209547268737109
COS with NEGATIVE:  0.014475731188221146


'negative'

In [14]:
demo_input = '''Even though Jordan Belfort is a convicted con man, Scorsese & DiCaprio were apparently too dense to realize that his book was yet another of his scams. They leapt on Belfort's book like pigs on a pile of slop and thought everyone would be just as enamored as they were with the alleged life story of a sociopathic, sleazeball swindler. One example of how they got conned: In real life, Belfort never warned his cohorts that he was wearing a wire! In reality, he was a cry-baby snitch who immediately agreed to testify against his co-conspirators, never giving any of them a word of warning.

It's rather amusing to watch Scorsese & DiCaprio as they try to explain that this movie isn't a glorification of Belfort's swindling & debauchery. C'mom, guys, 'fess up, all three hours of the movie is a celebration of it! You wasted $100 million to make an homage to a drug- addicted, misogynistic low-life who swindled people out of their hard- earned money! Proud of yourselves?

Here's the "Spoiler Alert": You are being conned if you go to this movie. But go ahead and give your hard-earned money to this convicted swindler and the "Hollywood sophisticates" who think his pathetically sad life is worth being turned into a movie. Just know that Belfort is laughing all the way to the bank, and you are his latest victim.'''

In [20]:
demo_nlp = nlp(demo_input)

In [22]:
demo_nlp.sentences[0].dependencies

[({
    "id": 9,
    "text": "man",
    "lemma": "man",
    "upos": "NOUN",
    "xpos": "NN",
    "feats": "Number=Sing",
    "head": 17,
    "deprel": "advcl",
    "misc": "start_char=46|end_char=49"
  },
  'advmod',
  {
    "id": 1,
    "text": "Even",
    "lemma": "even",
    "upos": "ADV",
    "xpos": "RB",
    "head": 9,
    "deprel": "advmod",
    "misc": "start_char=0|end_char=4"
  }),
 ({
    "id": 9,
    "text": "man",
    "lemma": "man",
    "upos": "NOUN",
    "xpos": "NN",
    "feats": "Number=Sing",
    "head": 17,
    "deprel": "advcl",
    "misc": "start_char=46|end_char=49"
  },
  'mark',
  {
    "id": 2,
    "text": "though",
    "lemma": "though",
    "upos": "SCONJ",
    "xpos": "IN",
    "head": 9,
    "deprel": "mark",
    "misc": "start_char=5|end_char=11"
  }),
 ({
    "id": 9,
    "text": "man",
    "lemma": "man",
    "upos": "NOUN",
    "xpos": "NN",
    "feats": "Number=Sing",
    "head": 17,
    "deprel": "advcl",
    "misc": "start_char=46|end_char=49"
  },

In [15]:
demo_input_vector = get_input_vector(demo_input, merged_sorted_dictionary)

sum_of_squares:  121


In [16]:
predict(demo_input_vector, positive_vector, negative_vector)

COS with POSITIVE:  0.01209547268737109
COS with NEGATIVE:  0.014475731188221146


'negative'

In [26]:
positive_fileids = movie_reviews.fileids('pos')
negative_fileids = movie_reviews.fileids('neg')
test_positive_ids = positive_fileids[501:750]
test_negative_ids = negative_fileids[501:750]
accuracy = 0
size = 500
iteration = 0
for test_id in test_positive_ids:
    stream = movie_reviews.open(test_id)
    review = stream.read()
    review_input_vector = get_input_vector(review, merged_sorted_dictionary)
    x = predict(review_input_vector, positive_vector, negative_vector)
    print('Iteration: ', iteration)
    iteration += 1
    if x == 'positive':
        print('Succeeded' + ' : ' + x)
        accuracy += 1
    else:
        print('Failed' + ' : ' + x)
        
for test_id in test_negative_ids:
    stream = movie_reviews.open(test_id)
    review = stream.read()
    review_input_vector = get_input_vector(review, merged_sorted_dictionary)
    x = predict(review_input_vector, positive_vector, negative_vector)
    print(iteration)
    iteration += 1
    if x == 'negative':
        print('Succeeded' + ' : ' + x)
        accuracy += 1
    else:
        print('Failed' + ' : ' + x)

sum_of_squares:  1843
COS with POSITIVE:  0.0320484029488591
COS with NEGATIVE:  0.030461897907354332
Iteration:  0
Succeeded : positive
sum_of_squares:  604
COS with POSITIVE:  0.021947108102446593
COS with NEGATIVE:  0.022912843379400475
Iteration:  1
Failed : negative
sum_of_squares:  955
COS with POSITIVE:  0.027039462115617127
COS with NEGATIVE:  0.0259399623042736
Iteration:  2
Succeeded : positive
sum_of_squares:  459
COS with POSITIVE:  0.02211088762698634
COS with NEGATIVE:  0.02199044892339242
Iteration:  3
Succeeded : positive
sum_of_squares:  994
COS with POSITIVE:  0.030498685227891464
COS with NEGATIVE:  0.02229943986905987
Iteration:  4
Succeeded : positive
sum_of_squares:  1427
COS with POSITIVE:  0.032217559309216735
COS with NEGATIVE:  0.028888370195493635
Iteration:  5
Succeeded : positive
sum_of_squares:  438
COS with POSITIVE:  0.020972812468449804
COS with NEGATIVE:  0.022421700623473644
Iteration:  6
Failed : negative
sum_of_squares:  921
COS with POSITIVE:  0.03

COS with NEGATIVE:  0.028187640337747347
Iteration:  59
Succeeded : positive
sum_of_squares:  320
COS with POSITIVE:  0.022482190693964156
COS with NEGATIVE:  0.018401259028034044
Iteration:  60
Succeeded : positive
sum_of_squares:  475
COS with POSITIVE:  0.01913602831977166
COS with NEGATIVE:  0.01825057394887234
Iteration:  61
Succeeded : positive
sum_of_squares:  1216
COS with POSITIVE:  0.02039479724161391
COS with NEGATIVE:  0.016553804286847527
Iteration:  62
Succeeded : positive
sum_of_squares:  236
COS with POSITIVE:  0.01657316717633686
COS with NEGATIVE:  0.01819300403061349
Iteration:  63
Failed : negative
sum_of_squares:  186
COS with POSITIVE:  0.0209772031845584
COS with NEGATIVE:  0.010598377644095144
Iteration:  64
Succeeded : positive
sum_of_squares:  381
COS with POSITIVE:  0.021398942618660488
COS with NEGATIVE:  0.016424538533665975
Iteration:  65
Succeeded : positive
sum_of_squares:  556
COS with POSITIVE:  0.024742953810737683
COS with NEGATIVE:  0.02322555267159

Iteration:  118
Succeeded : positive
sum_of_squares:  974
COS with POSITIVE:  0.024889291688999792
COS with NEGATIVE:  0.024633767320581704
Iteration:  119
Succeeded : positive
sum_of_squares:  760
COS with POSITIVE:  0.028844887613340694
COS with NEGATIVE:  0.026262877047213095
Iteration:  120
Succeeded : positive
sum_of_squares:  329
COS with POSITIVE:  0.020014956036416575
COS with NEGATIVE:  0.01885771152937301
Iteration:  121
Succeeded : positive
sum_of_squares:  621
COS with POSITIVE:  0.026069193095981737
COS with NEGATIVE:  0.022779583293687514
Iteration:  122
Succeeded : positive
sum_of_squares:  527
COS with POSITIVE:  0.024378680848125146
COS with NEGATIVE:  0.02193997820277031
Iteration:  123
Succeeded : positive
sum_of_squares:  302
COS with POSITIVE:  0.018697262262063655
COS with NEGATIVE:  0.01684011943231195
Iteration:  124
Succeeded : positive
sum_of_squares:  347
COS with POSITIVE:  0.01901772768757088
COS with NEGATIVE:  0.016116769116694137
Iteration:  125
Succeede

COS with NEGATIVE:  0.017222652576579085
Iteration:  177
Succeeded : positive
sum_of_squares:  524
COS with POSITIVE:  0.025401796407985903
COS with NEGATIVE:  0.021562605848050684
Iteration:  178
Succeeded : positive
sum_of_squares:  516
COS with POSITIVE:  0.022365881097433968
COS with NEGATIVE:  0.021067638548023918
Iteration:  179
Succeeded : positive
sum_of_squares:  265
COS with POSITIVE:  0.017650530868070762
COS with NEGATIVE:  0.018042659088460797
Iteration:  180
Failed : negative
sum_of_squares:  327
COS with POSITIVE:  0.023185381329975042
COS with NEGATIVE:  0.019330244631408783
Iteration:  181
Succeeded : positive
sum_of_squares:  1454
COS with POSITIVE:  0.024340545439648545
COS with NEGATIVE:  0.021628141956560855
Iteration:  182
Succeeded : positive
sum_of_squares:  385
COS with POSITIVE:  0.01927830017343382
COS with NEGATIVE:  0.01662064722969752
Iteration:  183
Succeeded : positive
sum_of_squares:  463
COS with POSITIVE:  0.020921253551824632
COS with NEGATIVE:  0.02

COS with POSITIVE:  0.024594986996872976
COS with NEGATIVE:  0.020297733966901883
Iteration:  236
Succeeded : positive
sum_of_squares:  726
COS with POSITIVE:  0.03144862780137928
COS with NEGATIVE:  0.020492141846797277
Iteration:  237
Succeeded : positive
sum_of_squares:  518
COS with POSITIVE:  0.022621286914785144
COS with NEGATIVE:  0.017128926870854995
Iteration:  238
Succeeded : positive
sum_of_squares:  465
COS with POSITIVE:  0.019443124707220594
COS with NEGATIVE:  0.02165010519550966
Iteration:  239
Failed : negative
sum_of_squares:  276
COS with POSITIVE:  0.01949388417831367
COS with NEGATIVE:  0.019217723638483414
Iteration:  240
Succeeded : positive
sum_of_squares:  270
COS with POSITIVE:  0.017550218714194002
COS with NEGATIVE:  0.016411161905637534
Iteration:  241
Succeeded : positive
sum_of_squares:  890
COS with POSITIVE:  0.022660509091606548
COS with NEGATIVE:  0.017504710902909677
Iteration:  242
Succeeded : positive
sum_of_squares:  453
COS with POSITIVE:  0.0216

COS with NEGATIVE:  0.0230783579570365
299
Succeeded : negative
sum_of_squares:  535
COS with POSITIVE:  0.02247853621390128
COS with NEGATIVE:  0.024012260186953627
300
Succeeded : negative
sum_of_squares:  410
COS with POSITIVE:  0.02165059411891857
COS with NEGATIVE:  0.022698632809345202
301
Succeeded : negative
sum_of_squares:  496
COS with POSITIVE:  0.019104414345327593
COS with NEGATIVE:  0.02009860506719591
302
Succeeded : negative
sum_of_squares:  223
COS with POSITIVE:  0.013149750781876314
COS with NEGATIVE:  0.015450690025767003
303
Succeeded : negative
sum_of_squares:  391
COS with POSITIVE:  0.018707748547397162
COS with NEGATIVE:  0.0228668017228098
304
Succeeded : negative
sum_of_squares:  497
COS with POSITIVE:  0.021513650685702185
COS with NEGATIVE:  0.021771221508105475
305
Succeeded : negative
sum_of_squares:  144
COS with POSITIVE:  0.012970498874329108
COS with NEGATIVE:  0.013212828151041698
306
Succeeded : negative
sum_of_squares:  306
COS with POSITIVE:  0.01

COS with POSITIVE:  0.013366537739518384
COS with NEGATIVE:  0.011564449966023028
364
Failed : positive
sum_of_squares:  364
COS with POSITIVE:  0.020159668119460836
COS with NEGATIVE:  0.023294908178303302
365
Succeeded : negative
sum_of_squares:  352
COS with POSITIVE:  0.01782029519813326
COS with NEGATIVE:  0.019966985940788266
366
Succeeded : negative
sum_of_squares:  824
COS with POSITIVE:  0.023689433897352074
COS with NEGATIVE:  0.030129657734443538
367
Succeeded : negative
sum_of_squares:  299
COS with POSITIVE:  0.01700515504098671
COS with NEGATIVE:  0.01893940572244493
368
Succeeded : negative
sum_of_squares:  1008
COS with POSITIVE:  0.020524410586776023
COS with NEGATIVE:  0.025927171187743625
369
Succeeded : negative
sum_of_squares:  596
COS with POSITIVE:  0.02080753791794298
COS with NEGATIVE:  0.018143647858300076
370
Failed : positive
sum_of_squares:  1170
COS with POSITIVE:  0.0280647142089936
COS with NEGATIVE:  0.034470442043254196
371
Succeeded : negative
sum_of_

428
Succeeded : negative
sum_of_squares:  430
COS with POSITIVE:  0.02051819482198605
COS with NEGATIVE:  0.021094135369638584
429
Succeeded : negative
sum_of_squares:  394
COS with POSITIVE:  0.020510203318792334
COS with NEGATIVE:  0.01914961471998563
430
Failed : positive
sum_of_squares:  577
COS with POSITIVE:  0.020302530501653787
COS with NEGATIVE:  0.023464105572070282
431
Succeeded : negative
sum_of_squares:  547
COS with POSITIVE:  0.020868992645874513
COS with NEGATIVE:  0.02386293575801025
432
Succeeded : negative
sum_of_squares:  271
COS with POSITIVE:  0.017965602410214786
COS with NEGATIVE:  0.0176166346941011
433
Failed : positive
sum_of_squares:  455
COS with POSITIVE:  0.020376304918038563
COS with NEGATIVE:  0.020176537621240588
434
Failed : positive
sum_of_squares:  592
COS with POSITIVE:  0.018697926467145944
COS with NEGATIVE:  0.023209761576089522
435
Succeeded : negative
sum_of_squares:  501
COS with POSITIVE:  0.01925526735590368
COS with NEGATIVE:  0.0214494107

COS with POSITIVE:  0.021793203160473455
COS with NEGATIVE:  0.026040854441014803
493
Succeeded : negative
sum_of_squares:  1075
COS with POSITIVE:  0.03163923488506075
COS with NEGATIVE:  0.025636342975291472
494
Failed : positive
sum_of_squares:  227
COS with POSITIVE:  0.018437536972007576
COS with NEGATIVE:  0.01812545714755547
495
Failed : positive
sum_of_squares:  305
COS with POSITIVE:  0.016379576603987516
COS with NEGATIVE:  0.019814760059114377
496
Succeeded : negative
sum_of_squares:  561
COS with POSITIVE:  0.022003120520616294
COS with NEGATIVE:  0.027486765751405035
497
Succeeded : negative


In [31]:
print('Prediction hits: ', accuracy)
accuracy = accuracy / size

Prediction hits:  456


In [32]:
accuracy

0.912